In [1]:
import sys
import os
sys.path.append('../')
print(os.getcwd())

import json
from agilkia import TraceSet
from sklearn.feature_extraction.text import CountVectorizer
from itertools import groupby
from spmf import Spmf
import numpy as np
import json
from os.path import exists
from pprint import pprint

/home/ftamagnan-unix/codevault/stvr_v3/exploration


In [2]:
from stvr.pattern_coverage import PatternCoverage
from utils_experiments import load_config_dict
from stvr.utils_preprocessing import *

/home/ftamagnan-unix/codevault/stvr_v3/exploration


In [3]:
datapath='../data/datasets/'
# dataset_name="femto_booking_agilkia_v6"
# dataset_name="femto_booking_with_drain_agilkia_v4"
# dataset_name="spree_5000_session_wo_responses_agilkia"
# dataset_name="teaming_execution"
# dataset_name="teaming_execution"
dataset_name="scanette_100043-steps"

config_dict=load_config_dict('../config/config.json')
config_dict_exp=config_dict['experiments']
freq=config_dict_exp[dataset_name]['freq']
traceset=load_traceset(datapath,dataset_name)
textset=traceset_to_textset(traceset,format='lst')

In [4]:
print(textset[0])

['<sos>', 'debloquer|0', 'scanner|-2', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'scanner|0', 'transmission|0', 'abandon|?', 'ouvrirSession|0', 'ajouter|0', 'fermerSession|0', 'payer|1', '<eos>']


In [5]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Download the nltk resources (if not already downloaded)
nltk.download('punkt')

# Load the Sentiment140 dataset (replace with the actual path)
# dataset_path = "../data/datasets/training.1600000.processed.noemoticon.csv"  # Provide the correct path
dataset_path = "../data/datasets/train.en"
# data = pd.read_csv(dataset_path, header=None, encoding='ISO-8859-1', names=['target', 'id', 'date', 'flag', 'user', 'text'])
with open(dataset_path, 'r', encoding='utf-8') as file:
    dataset = file.read().splitlines()

# Extract text from the dataset
# texts = data['text'].tolist()[0:5000]
texts=dataset[0:20000]
# Tokenize the text into lists of tokens
tokenized_texts = [word_tokenize(text) for text in texts]

# Display the first few tokenized texts as an example
for i in range(5):
    print(f"Text {i + 1}:", tokenized_texts[i])

[nltk_data] Downloading package punkt to /home/ftamagnan-
[nltk_data]     unix/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Text 1: ['iron', 'cement', 'is', 'a', 'ready', 'for', 'use', 'paste', 'which', 'is', 'laid', 'as', 'a', 'fillet', 'by', 'putty', 'knife', 'or', 'finger', 'in', 'the', 'mould', 'edges', '(', 'corners', ')', 'of', 'the', 'steel', 'ingot', 'mould', '.']
Text 2: ['iron', 'cement', 'protects', 'the', 'ingot', 'against', 'the', 'hot', ',', 'abrasive', 'steel', 'casting', 'process', '.']
Text 3: ['a', 'fire', 'restant', 'repair', 'cement', 'for', 'fire', 'places', ',', 'ovens', ',', 'open', 'fireplaces', 'etc', '.']
Text 4: ['Construction', 'and', 'repair', 'of', 'highways', 'and', '...']
Text 5: ['An', 'announcement', 'must', 'be', 'commercial', 'character', '.']


In [6]:
from collections import Counter
import numpy as np
def mining_metrics(sequences):

    # Calculate metrics
    num_samples = len(sequences)
    total_tokens = sum(len(seq) for seq in sequences)
    average_sequence_length = np.mean([len(seq) for seq in sequences])
    vocabulary = set(element for seq in sequences for element in seq)
    vocabulary_size = len(vocabulary)
    element_frequencies = Counter(element for seq in sequences for element in seq)
    average_element_frequency = np.mean(list(element_frequencies.values()))
    average_element_frequency_per_sample = average_element_frequency / num_samples
    average_element_frequency_per_token = average_element_frequency / total_tokens

    # Calculate maximum number of consecutive repetitions of a token
    max_consecutive_repetitions = 0
    k=5
    number_of_over_k_repetitions=0
    for seq in sequences:
        current_repetitions = 1
        more_than_k_rep=False
        for i in range(1, len(seq)):
            if seq[i] == seq[i - 1]:
                current_repetitions += 1
                max_consecutive_repetitions = max(max_consecutive_repetitions, current_repetitions)
                if current_repetitions>=k:
                    more_than_k_rep=True
            else:
                current_repetitions = 1
        if more_than_k_rep:
            number_of_over_k_repetitions+=1

    # Calculate average number of different tokens per sentence
    average_different_tokens_per_sentence = np.mean([len(set(seq)) for seq in sequences])



    metrics_dict={}
    def add_to(metrics_dict,name,value):
        metrics_dict[name]=value
        return metrics_dict
    
    metrics_dict=add_to(metrics_dict,"Number of Samples:", num_samples)
    
    metrics_dict=add_to(metrics_dict,"Total Tokens", total_tokens)
    metrics_dict=add_to(metrics_dict,"Average Sequence Length", average_sequence_length)
    metrics_dict=add_to(metrics_dict,"Vocabulary Size", vocabulary_size)
    metrics_dict=add_to(metrics_dict,"Average Element Frequency:", average_element_frequency)
    metrics_dict=add_to(metrics_dict,"Average Element Frequency per Sample", average_element_frequency_per_sample)
    metrics_dict=add_to(metrics_dict,"Average Element Frequency per Token", average_element_frequency_per_token)
    metrics_dict=add_to(metrics_dict,"Maximum Consecutive Repetitions of a Token", max_consecutive_repetitions)
    metrics_dict=add_to(metrics_dict,"Number of sample where there is more than "+str(k)+ " rep", number_of_over_k_repetitions)
    metrics_dict=add_to(metrics_dict,"Average Number of Different Tokens per Sentence", average_different_tokens_per_sentence)

    # Calculate rare element analysis (e.g., elements that appear only once)
    rare_elements = [element for element, freq in element_frequencies.items() if freq == 1]
    num_rare_elements = len(rare_elements)
    metrics_dict=add_to(metrics_dict,"Number of Rare Elements", num_rare_elements)
    metrics_dict=add_to(metrics_dict,"Share of rare elements with respect to all tokens",num_rare_elements/total_tokens)
    print("Rare Elements:", rare_elements)

    # Calculate longest and shortest sequences
    longest_sequence = max(sequences, key=len)
    shortest_sequence = min(sequences, key=len)
    metrics_dict=add_to(metrics_dict,"Longest Sequence length", len(longest_sequence))
    metrics_dict=add_to(metrics_dict,"Shortest Sequence length", len(shortest_sequence))

    # Calculate distribution of sequence lengths
    sequence_lengths = [len(seq) for seq in sequences]
    sequence_length_counts = Counter(sequence_lengths)
    print("Sequence Length Distribution:", sequence_length_counts)
    return metrics_dict

In [7]:
metrics=mining_metrics(tokenized_texts)
pprint(metrics)

Rare Elements: ['fillet', 'putty', 'knife', 'abrasive', 'casting', 'restant', 'ovens', 'P.O.Box', 'ALLOWED', 'Deliveries', 'Translator', 'Toolbar', 'pasge', 'TI', 'typing', 'relates', 'VTeacher', 'Launch', 'chewing', 'rebilled', '38,000', '19,800', 'Smartphones', 'thesaurus', 'wrapped', 'definable', 'idioms', '11,107', '9,800', '65,000', 'tighter', 'TIFF', 'Bringing', 'antiqued', 'swamped', 'varnish', 'destructively', 'divider', 'Palette', 'Layer', 'EPS', 'Freehand', 'compelling', 'InDesign', 'CS', 'XFL', 'DISCLAIMER', 'layouts', 'Sàrl', 'Arbitration', 'arbitrator', 'arbiter', 'Mediation', 'assists', 'expatriate', 'Haidegger', 'X1160', 'X1260', 'pencil', 'Convert', 'ArtWork', 'Chameleon', 'collage', 'underexposed', 'overexposed', 'SmartMask', 'Coloriage', 'colorizing', 'Buster', 'Magnifier', 'resizing', 'retouching', 'stains', 'reconstructs', 'lacking', 'sourrounding', 'Frames', 'chest', 'shocked', 'Akvis', 'tattoo', 'Rat', 'Formgebung', 'Deutschen', 'Industrie-', 'Handelskammertag', '

In [8]:
datapath='../data/datasets/'
# dataset_name="femto_booking_agilkia_v6"
# dataset_name="femto_booking_with_drain_agilkia_v4"
# dataset_name="spree_5000_session_wo_responses_agilkia"
# dataset_name="teaming_execution"
# dataset_name="teaming_execution"
dataset_name="scanette_100043-steps"

config_dict=load_config_dict('../config/config.json')
config_dict_exp=config_dict['experiments']
freq=config_dict_exp[dataset_name]['freq']
traceset=load_traceset(datapath,dataset_name)
scanner_textset=traceset_to_textset(traceset,format='lst')






In [9]:
datapath='../data/datasets/'
# dataset_name="femto_booking_agilkia_v6"
# dataset_name="femto_booking_with_drain_agilkia_v4"
# dataset_name="spree_5000_session_wo_responses_agilkia"
# dataset_name="teaming_execution"
dataset_name="teaming_execution"
# dataset_name="scanette_100043-steps"

config_dict=load_config_dict('../config/config.json')
config_dict_exp=config_dict['experiments']
freq=config_dict_exp[dataset_name]['freq']
traceset=load_traceset(datapath,dataset_name)
teaming_textset=traceset_to_textset(traceset,format='lst')






In [10]:
datapath='../data/datasets/'
# dataset_name="femto_booking_agilkia_v6"
# dataset_name="femto_booking_with_drain_agilkia_v4"
dataset_name="spree_5000_session_wo_responses_agilkia"
# dataset_name="teaming_execution"
# dataset_name="teaming_execution"
# dataset_name="scanette_100043-steps"

config_dict=load_config_dict('../config/config.json')
config_dict_exp=config_dict['experiments']
freq=config_dict_exp[dataset_name]['freq']
traceset=load_traceset(datapath,dataset_name)
spree_textset=traceset_to_textset(traceset,format='lst')





In [11]:
datapath='../data/datasets/'
dataset_name="femto_booking_agilkia_v6"
# dataset_name="femto_booking_with_drain_agilkia_v4"
# dataset_name="spree_5000_session_wo_responses_agilkia"
# dataset_name="teaming_execution"
# dataset_name="teaming_execution"
# dataset_name="scanette_100043-steps"

config_dict=load_config_dict('../config/config.json')
config_dict_exp=config_dict['experiments']
freq=config_dict_exp[dataset_name]['freq']
traceset=load_traceset(datapath,dataset_name)
femto_textset=traceset_to_textset(traceset,format='lst')
print(traceset[0])
print(femto_textset[0])





???
['<sos>', '/Web/reservation.phpclickbuttonbutton.reservationButtons||>||.pull-right-sm||>||.btn-success', '<eos>']


In [12]:

scanner_textset=[seq[1:-1] for seq in scanner_textset]
spree_textset_textset=[seq[1:-1] for seq in spree_textset]
teaming_textset=[seq[1:-1] for seq in teaming_textset]
femto_textset=[seq[1:-1] for seq in femto_textset]


print(femto_textset[10])

datasets=[(tokenized_texts,'sentiments140'),(scanner_textset,'scanner'),(spree_textset,'spree'),(teaming_textset,'teaming'),(femto_textset,'femto')]
metrics_datasets={}
for (dataset,dataset_name) in datasets:
    metrics_datasets[dataset_name]=mining_metrics(dataset)

table=''
table+="&".join(list(metrics_datasets.keys()))+'\\\\'+'\n'
table+="\hline"+'\n'
table+="\hline"+'\n'
for key in metrics_datasets['scanner']:
    table+=key+'&'
    for dataset_name in metrics_datasets:
        table+=str(round(metrics_datasets[dataset_name][key],2))+'&'
    table=table[:-1]
    table+='\\\\'+'\n'
print(table)



['/Web/schedule.phpclicka#scheduleListDropdown', '/Web/reservation.phpchangeselect#BeginPeriod', '/Web/reservation.phpchangeselect#EndPeriod', '/Web/reservation.phpchangeinputtext#reservationTitle']
Rare Elements: ['fillet', 'putty', 'knife', 'abrasive', 'casting', 'restant', 'ovens', 'P.O.Box', 'ALLOWED', 'Deliveries', 'Translator', 'Toolbar', 'pasge', 'TI', 'typing', 'relates', 'VTeacher', 'Launch', 'chewing', 'rebilled', '38,000', '19,800', 'Smartphones', 'thesaurus', 'wrapped', 'definable', 'idioms', '11,107', '9,800', '65,000', 'tighter', 'TIFF', 'Bringing', 'antiqued', 'swamped', 'varnish', 'destructively', 'divider', 'Palette', 'Layer', 'EPS', 'Freehand', 'compelling', 'InDesign', 'CS', 'XFL', 'DISCLAIMER', 'layouts', 'Sàrl', 'Arbitration', 'arbitrator', 'arbiter', 'Mediation', 'assists', 'expatriate', 'Haidegger', 'X1160', 'X1260', 'pencil', 'Convert', 'ArtWork', 'Chameleon', 'collage', 'underexposed', 'overexposed', 'SmartMask', 'Coloriage', 'colorizing', 'Buster', 'Magnifier'